In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import tensorflow as tf
import scipy
import glob
import sklearn 
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import layers, models, optimizers
from tensorflow.keras.layers import Input, Activation, Dense, LeakyReLU
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, Callback
from tensorflow.keras.optimizers import Adam, SGD
from keras_tuner import BayesianOptimization, HyperParameters

2024-01-23 14:46:44.417464: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-23 14:46:44.417526: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-23 14:46:44.419553: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-23 14:46:44.626224: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-23 14:46:46.572353: W tensorflow/compiler/tf2t

In [3]:
# Data import
wt_filtered = ['wt_filtered_lcc_3_50.lccdata', 'wt_filtered_lcc_12_50.lccdata', 'wt_filtered_lcc_20_50.lccdata']

# filtered wt LCC data import
wt_f_var_names = ['wt_3f', 'wt_12f', 'wt_20f']

for var, file in zip(wt_f_var_names, wt_filtered):
    globals()[var] = pd.read_csv(file, sep='\t').drop(columns='Unnamed: 0')

# filtered mutant LCC data import
D132H_filtered = ['D132H_filtered_lcc_3_50.lccdata', 'D132H_filtered_lcc_12_50.lccdata', 'D132H_filtered_lcc_20_50.lccdata']
D132H_f_var_names = ['D132H_3f', 'D132H_12f', 'D132H_20f']

for var, file in zip(D132H_f_var_names, D132H_filtered):
    globals()[var] = pd.read_csv(file, sep='\t').drop(columns='Unnamed: 0')
    
# Visualization of dataset
print('WT for window size = 3')
display(wt_3f)
print('WT for window size = 12')
display(wt_12f)
print('WT for window size = 20')
display(wt_20f)

print('\n')
print('---------------------------------')
print('D132H for window size = 3')
display(D132H_3f)
print('D132H for window size = 12')
display(D132H_12f)
print('D132H for window size = 20')
display(D132H_20f)

WT for window size = 3


,7,20,26,27
0,6.3249,9.6718,6.0082,9.6574
1,6.3105,10.6216,5.9996,11.2509
2,5.7233,11.4243,7.3021,11.5329
3,5.4864,10.7079,6.4156,10.7262
4,5.4673,12.0023,6.3573,11.2367
...,...,...,...,...
39995,4.9712,8.9677,10.1179,8.9430
39996,4.8021,9.0623,10.1026,8.6752
39997,5.9590,10.3438,8.6309,9.4556
39998,4.3996,10.3760,9.5901,9.3513


WT for window size = 12


,6,16,17,19,29,30
0,14.9702,17.3049,22.7152,30.7508,24.7567,23.1634
1,13.2704,22.4792,22.4227,28.1640,23.8363,23.2798
2,13.8551,25.5636,24.6903,30.2936,24.8655,23.4604
3,12.3881,26.4733,24.4458,29.5746,21.3842,22.6630
4,14.2788,24.0147,23.2922,27.5496,23.8249,22.3185
...,...,...,...,...,...,...
39995,19.7170,14.7258,12.9418,8.4287,24.3534,22.6400
39996,17.8741,15.5602,12.3054,7.1757,24.9417,24.1508
39997,17.2324,15.4532,12.3721,7.1490,23.8513,27.4276
39998,17.0120,15.1188,13.4625,7.5725,25.4096,26.8701


WT for window size = 20


...

,7,20,26,27
0,5.6253,11.4558,7.0366,10.8265
1,5.6219,11.7791,6.8272,10.8156
2,6.3355,9.9860,6.2585,9.7506
3,4.7119,8.4264,5.8994,8.1239
4,7.7166,9.3508,6.4404,10.1599
...,...,...,...,...
39995,4.6642,10.3721,10.9673,9.8788
39996,4.7521,11.6737,10.7638,9.5196
39997,4.5160,9.7824,10.9669,9.6701
39998,4.8358,7.8161,10.7679,9.6355


D132H for window size = 12


,6,16,17,19,29,30
0,14.9958,16.3721,20.6416,29.6048,24.1730,23.9425
1,15.0499,15.4543,20.4593,28.2314,25.1477,23.0454
2,16.5011,13.1972,16.8349,29.2049,23.6181,18.1444
3,13.6412,15.6381,18.6334,27.5069,25.6890,20.3013
4,16.5941,18.8374,20.8179,29.7816,23.7772,20.6936
...,...,...,...,...,...,...
39995,13.3469,15.4144,13.0795,12.4362,7.2937,12.2014
39996,13.3072,15.0391,12.4597,12.2133,7.2107,13.3563
39997,13.4729,15.5238,12.5300,12.2077,7.7544,12.6172
39998,12.8318,15.9725,13.9111,12.2538,7.3585,12.1032


D132H for window size = 20


,9,21
0,13.4055,41.6856
1,12.0150,40.2262
2,17.0081,41.7220
3,15.6520,40.4408
4,20.7450,38.0932
...,...,...
39995,14.0324,23.2605
39996,14.9455,22.4160
39997,14.0516,22.7481
39998,14.0350,23.5958


In [4]:
# Concateneate wt and mutant dataframes and rename columns

wt_f = pd.concat([wt_3f, wt_12f, wt_20f], axis = 1)
    
D132H_f = pd.concat([D132H_3f, D132H_12f, D132H_20f], axis = 1)

colnames = [*range(0,12)]
colnames
wt_f.columns = colnames
D132H_f.columns = colnames

In [5]:
# Data pre processing

def preprocessing(wt, mutant):
    
    wt_label = np.zeros(len(wt)) # Set wt labels to 0
    
    mutant_label = np.ones(len(mutant))
    
    # Concatenate data frames and label arrays

    X_train_full = pd.concat([wt.reset_index(), mutant.reset_index()])
    y_train_full = np.concatenate((wt_label, mutant_label))
    
    #Drop index column and normalise training data
    X_train_full = X_train_full.drop(columns = 'index')
    
    X_train_full= X_train_full.div(100) ## When changed from 100 to 56.1035, errors generated.
    
    # Separate training and validation sets and print relevant shapes
    X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, stratify=y_train_full, test_size=0.2)
    
    print(X_train.shape)
    print(X_valid.shape)
    print(y_train.shape)
    print(y_valid.shape)
    
    return X_train, X_valid, y_train, y_valid

In [6]:
X_train_f, X_valid_f, y_train_f, y_valid_f = preprocessing(wt_f, D132H_f)

(64000, 12)
(16000, 12)
(64000,)
(16000,)


In [ ]:
X_valid_f.to_csv('X_val.csv')

y_valid_f_df = pd.DataFrame({'class':y_valid_f})
y_valid_f_df.to_csv('y_val_df.csv')

In [7]:
print(X_valid_f)

              0         1         2         3         4         5         6  \
30165  0.048119  0.101472  0.091832  0.107898  0.145325  0.291379  0.282538   
38538  0.068959  0.116012  0.106705  0.091525  0.127055  0.203078  0.162565   
2022   0.095933  0.045604  0.077096  0.096224  0.171806  0.146170  0.170079   
27634  0.045609  0.082652  0.079041  0.121000  0.098173  0.175467  0.155925   
26385  0.058116  0.070519  0.084140  0.091053  0.123414  0.103483  0.051177   
...         ...       ...       ...       ...       ...       ...       ...   
5720   0.054092  0.078507  0.089275  0.099536  0.130755  0.130880  0.126251   
13949  0.050674  0.090001  0.083034  0.101010  0.153621  0.133054  0.081998   
29852  0.062419  0.083368  0.082029  0.078645  0.186324  0.133922  0.170336   
21815  0.046261  0.089445  0.052040  0.091658  0.145514  0.143408  0.161470   
24882  0.049735  0.104655  0.093851  0.091000  0.110423  0.151178  0.132919   

              7         8         9        10      

In [6]:
# Get autoencoder model
def get_ae(train_data, LeReLU_alpha=0.01):
    
    #Input layer
    input_layer = Input(shape=(train_data.shape[1]), name='ae_input')
    
    encoder = Dense(336, activation=LeakyReLU(alpha=LeReLU_alpha), name='e1')(input_layer)
    encoder = Dense(208, activation=LeakyReLU(alpha=LeReLU_alpha), name='e2')(encoder)
    encoder = Dense(240, activation=LeakyReLU(alpha=LeReLU_alpha), name='e3')(encoder)

    encoded = Dense(2, activation=LeakyReLU(alpha=LeReLU_alpha), name='ae_latent')(encoder)
    
    decoder = Dense(240, activation=LeakyReLU(alpha=LeReLU_alpha), name='d1')(encoded)
    decoder = Dense(208, activation=LeakyReLU(alpha=LeReLU_alpha), name='d2')(decoder)
    decoder = Dense(336, activation=LeakyReLU(alpha=LeReLU_alpha), name='d3')(decoder)

    output_layer = Dense(train_data.shape[1], activation=LeakyReLU(alpha=LeReLU_alpha), name='ae_output')(decoder)
    
    model = Model(input_layer, output_layer)

    return model

In [7]:
# Get ae for filtered data
autoencoder = get_ae(X_train_f)

2024-01-23 12:45:07.188508: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [8]:
# Print summary of ae model
autoencoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 ae_input (InputLayer)       [(None, 12)]              0         
                                                                 
 e1 (Dense)                  (None, 336)               4368      
                                                                 
 e2 (Dense)                  (None, 208)               70096     
                                                                 
 e3 (Dense)                  (None, 240)               50160     
                                                                 
 ae_latent (Dense)           (None, 2)                 482       
                                                                 
 d1 (Dense)                  (None, 240)               720       
                                                                 
 d2 (Dense)                  (None, 208)               50128 

In [9]:
# Compile the model
autoencoder.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=tf.keras.optimizers.Adam(learning_rate = 0.00001))

In [10]:
## Template to copy and paste
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
import numpy as np
import os 

# New folder name
folder_name = "AET_CF_Trial_11"

# Check if the folder exists. If not, create it.
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

for counts in tqdm(range(2)):  # this determines the number of epoch sets
    # Open a file for this epoch set to write outputs
    name = "11_LSP_AET_CF_"
    output_path = os.path.join(folder_name, f"11_LSP_AET_CF_Predictions_{counts}.txt")
    with open(output_path, "w") as file:
        # Autoencoder training
        history = autoencoder.fit(X_train_f, X_train_f, batch_size=256,
                                  epochs=1000, validation_data=(X_valid_f, X_valid_f), verbose=1)
        
        # Convert history object to dataframe and plot rates
        training_history = pd.DataFrame(history.history)
        plt.plot(training_history)
        file_name_0 = os.path.join(folder_name, name + "Training_History" + str(counts))
        training_history.to_pickle(file_name_0)
        file_name_1 = os.path.join(folder_name, name + "Training_History" + str(counts) + ".png")
        plt.savefig(file_name_1, dpi=300)
        plt.clf()

        # Read in latent layer
        dr_model = tf.keras.models.Model(inputs=autoencoder.get_layer('ae_input').input,
                                         outputs=autoencoder.get_layer('ae_latent').output)
        # Write the model summary to the file
        dr_model.summary(print_fn=lambda x: file.write(x + '\n'))

        # Define batch size for processing validation data
        batch_size = 32

        # Initialize lists to hold prediction results
        x = []
        y = []
        z = []

        # Process validation data in batches
        for batch_start in range(0, len(X_valid_f), batch_size):
            batch_end = min(batch_start + batch_size, len(X_valid_f))
            X_batch = np.array(X_valid_f.iloc[batch_start:batch_end])
            y_batch = y_valid_f[batch_start:batch_end]

            # Make predictions for this batch
            op_batch = dr_model.predict(X_batch, verbose=0)

            # Process and store the results
            for i, op in enumerate(op_batch):
                z.append(y_batch[i])
                x.append(op[0])
                y.append(op[1])
                file.write(f"Prediction {batch_start + i}: {op}\n")

        # Construct and save the data frame
        df = pd.DataFrame()
        df['x'] = x
        df['y'] = y
        df['z'] = ["trajectory-" + str(k) for k in z]

        plt.figure(figsize=(8, 6))
        fig = sns.scatterplot(x='x', y='y', hue='z', data=df, s=10)
        file_name_2 = os.path.join(folder_name, name + str(counts) + ".png")
        fig.figure.savefig(file_name_2, dpi=300)
        plt.clf()

        file_name_3 = os.path.join(folder_name, name + 'Predictions' + str(counts))
        df.to_pickle(file_name_3)

        # Save the model in a subfolder within Trial folder
        model_folder = os.path.join(folder_name, 'models')
        if not os.path.exists(model_folder):
            os.makedirs(model_folder)
        file_name = os.path.join(model_folder, 'saved_model_' + name + str(counts))
        autoencoder.save(file_name)

  0%|                                                     | 0/2 [00:00<?, ?it/s]

Epoch 1/1000
250/250 [==============================] - 3s 8ms/step - loss: 0.0176 - val_loss: 0.0099
Epoch 2/1000
250/250 [==============================] - 2s 6ms/step - loss: 0.0082 - val_loss: 0.0058
Epoch 3/1000
250/250 [==============================] - 2s 6ms/step - loss: 0.0026 - val_loss: 0.0015
Epoch 4/1000
250/250 [==============================] - 2s 8ms/step - loss: 0.0015 - val_loss: 0.0014
Epoch 5/1000
250/250 [==============================] - 1s 5ms/step - loss: 0.0014 - val_loss: 0.0014
Epoch 6/1000
250/250 [==============================] - 1s 5ms/step - loss: 0.0013 - val_loss: 0.0012
Epoch 7/1000
250/250 [==============================] - 1s 5ms/step - loss: 0.0010 - val_loss: 8.7810e-04
Epoch 8/1000
250/250 [==============================] - 1s 5ms/step - loss: 8.1928e-04 - val_loss: 7.8510e-04
Epoch 9/1000
250/250 [==============================] - 1s 5ms/step - loss: 7.6793e-04 - val_loss: 7.5238e-04
Epoch 10/1000
250/250 [==============================] - 1s 5m

250/250 [==============================] - 1s 5ms/step - loss: 4.5774e-04 - val_loss: 4.5865e-04
Epoch 149/1000
250/250 [==============================] - 1s 5ms/step - loss: 4.5729e-04 - val_loss: 4.5786e-04
Epoch 150/1000
250/250 [==============================] - 1s 5ms/step - loss: 4.5677e-04 - val_loss: 4.5760e-04
Epoch 151/1000
250/250 [==============================] - 1s 5ms/step - loss: 4.5632e-04 - val_loss: 4.5713e-04
Epoch 152/1000
250/250 [==============================] - 1s 5ms/step - loss: 4.5563e-04 - val_loss: 4.5646e-04
Epoch 153/1000
250/250 [==============================] - 1s 5ms/step - loss: 4.5534e-04 - val_loss: 4.5609e-04
Epoch 154/1000
250/250 [==============================] - 1s 5ms/step - loss: 4.5486e-04 - val_loss: 4.5590e-04
Epoch 155/1000
250/250 [==============================] - 1s 5ms/step - loss: 4.5424e-04 - val_loss: 4.5492e-04
Epoch 156/1000
250/250 [==============================] - 1s 5ms/step - loss: 4.5393e-04 - val_loss: 4.5497e-04
Epoch 1

250/250 [==============================] - 1s 5ms/step - loss: 4.1710e-04 - val_loss: 4.1717e-04
Epoch 295/1000
250/250 [==============================] - 1s 5ms/step - loss: 4.1697e-04 - val_loss: 4.1670e-04
Epoch 296/1000
250/250 [==============================] - 1s 5ms/step - loss: 4.1672e-04 - val_loss: 4.1705e-04
Epoch 297/1000
250/250 [==============================] - 1s 5ms/step - loss: 4.1640e-04 - val_loss: 4.1747e-04
Epoch 298/1000
250/250 [==============================] - 1s 5ms/step - loss: 4.1622e-04 - val_loss: 4.1659e-04
Epoch 299/1000
250/250 [==============================] - 1s 5ms/step - loss: 4.1634e-04 - val_loss: 4.1604e-04
Epoch 300/1000
250/250 [==============================] - 1s 5ms/step - loss: 4.1616e-04 - val_loss: 4.1587e-04
Epoch 301/1000
250/250 [==============================] - 1s 5ms/step - loss: 4.1575e-04 - val_loss: 4.1557e-04
Epoch 302/1000
250/250 [==============================] - 1s 5ms/step - loss: 4.1574e-04 - val_loss: 4.1541e-04
Epoch 3

250/250 [==============================] - 1s 5ms/step - loss: 3.9480e-04 - val_loss: 3.9634e-04
Epoch 441/1000
250/250 [==============================] - 1s 5ms/step - loss: 3.9459e-04 - val_loss: 3.9638e-04
Epoch 442/1000
250/250 [==============================] - 1s 5ms/step - loss: 3.9472e-04 - val_loss: 3.9580e-04
Epoch 443/1000
250/250 [==============================] - 1s 5ms/step - loss: 3.9447e-04 - val_loss: 3.9641e-04
Epoch 444/1000
250/250 [==============================] - 1s 5ms/step - loss: 3.9435e-04 - val_loss: 3.9707e-04
Epoch 445/1000
250/250 [==============================] - 1s 5ms/step - loss: 3.9455e-04 - val_loss: 3.9588e-04
Epoch 446/1000
250/250 [==============================] - 1s 5ms/step - loss: 3.9407e-04 - val_loss: 3.9570e-04
Epoch 447/1000
250/250 [==============================] - 1s 5ms/step - loss: 3.9412e-04 - val_loss: 3.9574e-04
Epoch 448/1000
250/250 [==============================] - 1s 5ms/step - loss: 3.9395e-04 - val_loss: 3.9544e-04
Epoch 4

...

250/250 [==============================] - 1s 5ms/step - loss: 3.2200e-04 - val_loss: 3.3010e-04
Epoch 952/1000
250/250 [==============================] - 1s 5ms/step - loss: 3.2173e-04 - val_loss: 3.2957e-04
Epoch 953/1000
250/250 [==============================] - 1s 5ms/step - loss: 3.2207e-04 - val_loss: 3.2964e-04
Epoch 954/1000
250/250 [==============================] - 1s 5ms/step - loss: 3.2171e-04 - val_loss: 3.2944e-04
Epoch 955/1000
250/250 [==============================] - 1s 5ms/step - loss: 3.2241e-04 - val_loss: 3.2905e-04
Epoch 956/1000
250/250 [==============================] - 1s 5ms/step - loss: 3.2175e-04 - val_loss: 3.2971e-04
Epoch 957/1000
250/250 [==============================] - 1s 5ms/step - loss: 3.2202e-04 - val_loss: 3.3145e-04
Epoch 958/1000
250/250 [==============================] - 1s 5ms/step - loss: 3.2176e-04 - val_loss: 3.2922e-04
Epoch 959/1000
250/250 [==============================] - 1s 5ms/step - loss: 3.2181e-04 - val_loss: 3.2957e-04
Epoch 9

INFO:tensorflow:Assets written to: AET_CF_Trial_11/models/saved_model_11_LSP_AET_CF_1/assets
100%|███████████████████████████████████████████| 2/2 [43:07<00:00, 1293.67s/it]


<Figure size 640x480 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

In [11]:
import numpy as np

# Save index list
shuffled_index_train = X_train_f.index.to_list()
shuffled_index_valid = X_valid_f.index.to_list()

# Save shuffled indices of the training data
with open(f"{folder_name}/shufftrain.npy", "wb") as file:
    np.save(file, shuffled_index_train)

# Save shuffled indices of the validation data
with open(f"{folder_name}/shuffval.npy", "wb") as file:
    np.save(file, shuffled_index_valid)
